In [1]:
import numpy as np
import pandas as pd

In [2]:
image = pd.read_csv('../raw_data/train_image_level.csv')
study = pd.read_csv('../raw_data/train_study_level.csv')
df_image = image.copy()
df_study = study.copy()

In [3]:
print(df_image.shape)
df_image.head()

(6334, 4)


,id,boxes,label,StudyInstanceUID
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e


In [4]:
print(df_study.shape)
df_study.head()

(6054, 5)


,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,00086460a852_study,0,1,0,0
1,000c9c05fd14_study,0,0,0,1
2,00292f8c37bd_study,1,0,0,0
3,005057b3f880_study,1,0,0,0
4,0051d9b12e72_study,0,0,0,1


In [5]:
df_study[['Negative for Pneumonia','Typical Appearance','Indeterminate Appearance','Atypical Appearance']].value_counts()

Negative for Pneumonia  Typical Appearance  Indeterminate Appearance  Atypical Appearance
0                       1                   0                         0                      2855
1                       0                   0                         0                      1676
0                       0                   1                         0                      1049
                                            0                         1                       474
dtype: int64

## Etude et merge des DF

In [6]:
df_image.isna().sum()

id                     0
boxes               2040
label                  0
StudyInstanceUID       0
dtype: int64

In [7]:
df_image['id'].duplicated().sum()

0

In [8]:
df_image['StudyInstanceUID'].duplicated().sum()

280

**/!\ Il y a 280 études** (StudyInstanceUID) **pour lesquels il y a 2 images dans l'étude**

In [9]:
df_study.isnull().sum()

id                          0
Negative for Pneumonia      0
Typical Appearance          0
Indeterminate Appearance    0
Atypical Appearance         0
dtype: int64

In [10]:
def clear_id_image(image_id): 
    image_id = image_id.replace('_image','')
    return image_id

def clear_id_study(study_id): 
    study_id = study_id.replace('_study','')
    return study_id

In [11]:
df_image['image_id'] = df_image['id'].apply(clear_id_image)
df_image.head()

,id,boxes,label,StudyInstanceUID,image_id
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,000a312787f2
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed,000c3a3f293f
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,0012ff7358bc
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,001398f4ff4f
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,001bd15d1891


In [12]:
df_study['StudyInstanceUID'] = df_study['id'].apply(clear_id_study)
df_study.head()

,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,StudyInstanceUID
0,00086460a852_study,0,1,0,0,00086460a852
1,000c9c05fd14_study,0,0,0,1,000c9c05fd14
2,00292f8c37bd_study,1,0,0,0,00292f8c37bd
3,005057b3f880_study,1,0,0,0,005057b3f880
4,0051d9b12e72_study,0,0,0,1,0051d9b12e72


In [13]:
df = (df_image.drop(columns='id')).merge(df_study.drop(columns='id'), on ='StudyInstanceUID', how='left')
print(df.shape)
df.head()

(6334, 8)


,boxes,label,StudyInstanceUID,image_id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,000a312787f2,0,1,0,0
1,NaN,none 1 0 0 1 1,ff0879eb20ed,000c3a3f293f,1,0,0,0
2,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,0012ff7358bc,0,1,0,0
3,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,001398f4ff4f,0,0,0,1
4,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,001bd15d1891,0,1,0,0


In [14]:
df.isna().sum()

boxes                       2040
label                          0
StudyInstanceUID               0
image_id                       0
Negative for Pneumonia         0
Typical Appearance             0
Indeterminate Appearance       0
Atypical Appearance            0
dtype: int64

In [15]:
df[['Negative for Pneumonia','Typical Appearance','Indeterminate Appearance','Atypical Appearance']].value_counts().sum()

6334

In [16]:
df[['Negative for Pneumonia','Typical Appearance','Indeterminate Appearance','Atypical Appearance']].value_counts()

Negative for Pneumonia  Typical Appearance  Indeterminate Appearance  Atypical Appearance
0                       1                   0                         0                      3007
1                       0                   0                         0                      1736
0                       0                   1                         0                      1108
                                            0                         1                       483
dtype: int64

## Dataframe simplifié pour tâche de classification seule

In [17]:
df_simple = df.drop(columns=['boxes','label'])
df_simple.head()

,StudyInstanceUID,image_id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,5776db0cec75,000a312787f2,0,1,0,0
1,ff0879eb20ed,000c3a3f293f,1,0,0,0
2,9d514ce429a7,0012ff7358bc,0,1,0,0
3,28dddc8559b2,001398f4ff4f,0,0,0,1
4,dfd9fdd85a3e,001bd15d1891,0,1,0,0


In [18]:
df_simple.shape

(6334, 6)

In [19]:
df_simple.isna().sum()

StudyInstanceUID            0
image_id                    0
Negative for Pneumonia      0
Typical Appearance          0
Indeterminate Appearance    0
Atypical Appearance         0
dtype: int64

## Utilisation de la colonne 'boxes'

### Traitement des NaN

In [20]:
df_image['boxes'].isna().sum()

2040

In [21]:
df_nan = df.loc[df_image['boxes'].isna() == True ]
print(df_nan[['Negative for Pneumonia','Typical Appearance','Indeterminate Appearance','Atypical Appearance']].value_counts())
print(df_nan['label'].value_counts())
df_nan.head(5)

Negative for Pneumonia  Typical Appearance  Indeterminate Appearance  Atypical Appearance
1                       0                   0                         0                      1736
0                       1                   0                         0                       153
                        0                   0                         1                        92
                                            1                         0                        59
dtype: int64
none 1 0 0 1 1    2040
Name: label, dtype: int64


,boxes,label,StudyInstanceUID,image_id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
1,NaN,none 1 0 0 1 1,ff0879eb20ed,000c3a3f293f,1,0,0,0
6,NaN,none 1 0 0 1 1,2fa400b873f5,0023f02ae886,1,0,0,0
22,NaN,none 1 0 0 1 1,1a58b43cf286,00c1515729a8,0,1,0,0
24,NaN,none 1 0 0 1 1,74ba8f2badcb,00e3a7e91a34,1,0,0,0
26,NaN,none 1 0 0 1 1,2a7b4e669dda,010214f9b600,1,0,0,0


**/!\ Les 1617 'Negative for Pneumonia' ont tous NaN dans la colonne 'boxes', mais pas tous les NaN correspondent à 'Negative for Pneumonia'**

Aussi, toutes les boxes NaN ont un label associé 'none 1 0 0 1 1'

### Exportation des infos

## Fusion avec les datas images